# Optimal Sparse Steering via Convex Optimization

This notebook demonstrates two convex-optimization formulations for finding the **sparsest SAE feature intervention** that steers model behavior:

- **SOCP** (Second-Order Cone Program): minimizes $\ell_1$ subject to a hard $\ell_2$ coherence constraint $\|D^\top \delta\|_2 \leq \epsilon$.
- **QP** (Quadratic Program): minimizes $\ell_1 + \frac{\lambda}{2}\|D^\top \delta\|_2^2$, supporting warm-starting across sequential solves.

In [ ]:
# @title Setup
import sys, os

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    REPO_DIR = '/content/optimal_sparse_steering'
    if os.path.exists(REPO_DIR):
        !git -C {REPO_DIR} pull -q
    else:
        !git clone -q https://github.com/tgautam23/optimal_sparse_steering.git {REPO_DIR}
    !pip install -q torch transformer-lens sae-lens cvxpy datasets scikit-learn matplotlib seaborn tqdm transformers 2>&1 | grep -v "dependency conflicts\|incompatible\|pip's dependency resolver"
    PROJECT_ROOT = REPO_DIR
    try:
        import numpy as _np; import seaborn as _sns; del _np, _sns
    except (ValueError, ImportError):
        print("Restarting runtime for numpy compatibility...")
        os.kill(os.getpid(), 9)
else:
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

sys.path.insert(0, PROJECT_ROOT)

import logging
import time
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from configs.base import ExperimentConfig, ModelConfig, DataConfig
from src.models.wrapper import ModelWrapper
from src.models.sae_utils import get_decoder_matrix
from src.data.loader import load_dataset_splits
from src.data.preprocessing import extract_activations, extract_sae_features
from src.probes.linear_probe import LinearProbe
from src.steering.convex_optimal import ConvexOptimalSteering
from src.steering.qp_optimal import QPOptimalSteering
from src.evaluation.metrics import compute_probe_score, compute_l0

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(name)s: %(message)s')
sns.set_theme(style="whitegrid")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Part 1: Sentiment Steering (GPT-2 Small + SST-2)

In [ ]:
# @title Load Model, Data, Probe, and Decoder Matrix
config = ExperimentConfig()
config.model.device = device
config.data.split = "train"
config.data.max_samples_per_class = 500

# Load model
model_wrapper = ModelWrapper(config.model)
layer = config.model.steering_layer
print(f"Model: {config.model.tl_name}, d_model={config.model.d_model}, steering_layer={layer}")

# Load SAE and get decoder matrix
sae = model_wrapper.get_sae(layer)
D = get_decoder_matrix(sae)
print(f"SAE decoder matrix D shape: {D.shape}")

# Load dataset splits
data = load_dataset_splits(config.data)
print(f"Train: {len(data['train_texts'])}, Test: {len(data['test_texts'])}")

# Extract activations at the steering layer
train_acts = extract_activations(data['train_texts'], model_wrapper, layer, batch_size=config.model.batch_size)
test_acts = extract_activations(data['test_texts'], model_wrapper, layer, batch_size=config.model.batch_size)

train_np = train_acts.numpy()
test_np = test_acts.numpy()
train_labels = np.array(data['train_labels'])
test_labels = np.array(data['test_labels'])

# Train linear probe with C=0.01 for better regularization
probe = LinearProbe(d_model=config.model.d_model, C=0.01)
probe.fit(train_np, train_labels)
print(f"Probe accuracy -- train: {probe.score(train_np, train_labels):.4f}, "
      f"test: {probe.score(test_np, test_labels):.4f}")

# Extract probe parameters
probe_w = probe.weight_vector
probe_b = probe.bias
target_class = 1
print(f"Probe weight norm: {np.linalg.norm(probe_w):.4f}, bias: {probe_b:.4f}")

### Neutral Prompts & Helper Functions

In [ ]:
# @title Define Prompts and Helper Functions
sentiment_prompts = [
    "The movie was",
    "I think the food at this restaurant is",
    "After reading the book, I felt that it",
    "The weather today seems",
    "My experience with the new product has been",
]

alpha_values = [0, 1, 5, 10]


def compute_generation_probe_score(text, model_wrapper, probe, layer, target_class=1):
    """Compute probe score on generated text (unsteered forward pass)."""
    tokenizer = model_wrapper.tokenizer
    model = model_wrapper.model
    device_m = next(model.parameters()).device
    hook_name = f"blocks.{layer}.hook_resid_pre"

    input_ids = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=128).to(device_m)
    with torch.no_grad():
        _, cache = model_wrapper.run_with_cache(input_ids, names_filter=hook_name)
    h = cache[hook_name][0, -1, :].cpu().numpy().reshape(1, -1)
    score = probe.predict_proba(h)[0, target_class]
    return float(score)


def run_steering_experiment(prompts, method, model_wrapper, layer, probe, probe_w, probe_b,
                            D, target_class, alpha_values, max_new_tokens=50):
    """Solve once per prompt, generate at multiple alpha values, return results."""
    model = model_wrapper.model
    tokenizer = model_wrapper.tokenizer
    hook_name = model_wrapper.get_hook_name(layer)
    device_m = next(model.parameters()).device

    all_results = []
    all_active_features = []

    for prompt in prompts:
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device_m)

        # Extract last-token activation + SAE features
        with torch.no_grad():
            _, cache = model_wrapper.run_with_cache(input_ids, names_filter=hook_name)
        h = cache[hook_name][0, -1, :].cpu()
        sae_feats = model_wrapper.encode_with_sae(h.unsqueeze(0), layer).squeeze(0)

        # Solve optimization once
        t0 = time.time()
        sv = method.compute_steering_vector(
            h=h, probe_w=probe_w, probe_b=probe_b,
            D=D, sae_features=sae_feats, target_class=target_class,
        )
        solve_time = getattr(method, '_solve_time', None)
        if solve_time is None:
            solve_time = time.time() - t0

        active_feats = method.active_features.tolist() if hasattr(method, 'active_features') and method._active_features is not None else []
        l0 = int((method.delta > 1e-6).sum()) if hasattr(method, 'delta') and method._delta is not None else 0
        all_active_features.append(set(active_feats))

        prompt_results = {"prompt": prompt, "solve_time": solve_time, "l0": l0, "active_features": active_feats, "generations": {}}

        sv_device = sv.to(device_m)

        for alpha in alpha_values:
            if alpha == 0:
                # Unsteered generation
                def hook_fn(acts, hook):
                    return acts
            else:
                def hook_fn(acts, hook, _sv=sv_device, _alpha=alpha):
                    return acts + _alpha * _sv

            model.add_hook(hook_name, hook_fn)
            try:
                with torch.no_grad():
                    output_ids = model.generate(
                        input_ids, max_new_tokens=max_new_tokens,
                        temperature=0.7, top_p=0.9, do_sample=True,
                    )
            finally:
                model.reset_hooks()

            gen_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            probe_score = compute_generation_probe_score(gen_text, model_wrapper, probe, layer, target_class)

            prompt_results["generations"][alpha] = {
                "text": gen_text,
                "probe_score": probe_score,
            }

        all_results.append(prompt_results)

    return all_results, all_active_features


def display_results(results, method_name):
    """Print results in a readable format."""
    print(f"\n{'='*80}")
    print(f"{method_name}")
    print(f"{'='*80}")
    for r in results:
        print(f"\nPrompt: \"{r['prompt']}\"")
        print(f"  Solve time: {r['solve_time']:.3f}s | L0: {r['l0']} active features")
        for alpha, gen in r['generations'].items():
            score_str = f"{gen['probe_score']:.4f}"
            text_preview = gen['text'][:120].replace('\n', ' ')
            print(f"  alpha={alpha:2d} | P(target)={score_str} | {text_preview}...")

### SOCP Steering
Solve the SOCP (hard $\ell_2$ coherence constraint $\|D^\top \delta\|_2 \leq \epsilon$) once per prompt, then generate at each alpha.

In [ ]:
# @title SOCP Steering -- Sentiment
socp = ConvexOptimalSteering(epsilon=5.0, tau=0.5, solver="SCS", prefilter_topk=2000)

socp_results, socp_features = run_steering_experiment(
    sentiment_prompts, socp, model_wrapper, layer, probe, probe_w, probe_b,
    D, target_class=1, alpha_values=alpha_values,
)

display_results(socp_results, "SOCP (epsilon=5.0)")

### QP Steering (with Warm-Start)
Solve the QP ($\ell_1$ + $\frac{\lambda}{2}\|D^\top \delta\|_2^2$ penalty) with warm-starting across prompts.

In [ ]:
# @title QP Steering -- Sentiment
qp = QPOptimalSteering(lam=1.0, tau=0.5, solver="SCS", prefilter_topk=2000, warm_start=True)

qp_results, qp_features = run_steering_experiment(
    sentiment_prompts, qp, model_wrapper, layer, probe, probe_w, probe_b,
    D, target_class=1, alpha_values=alpha_values,
)

display_results(qp_results, "QP (lambda=1.0, warm-start)")

### Probe Score Comparison

In [ ]:
# @title Probe Scores: SOCP vs QP across Alpha
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, (name, results) in zip(axes, [("SOCP", socp_results), ("QP", qp_results)]):
    for r in results:
        scores = [r['generations'][a]['probe_score'] for a in alpha_values]
        ax.plot(alpha_values, scores, marker='o', label=r['prompt'][:30] + "...")
    ax.set_xlabel('Alpha')
    ax.set_ylabel('P(positive sentiment)')
    ax.set_title(f'{name}: Probe Score vs Alpha')
    ax.legend(fontsize=8)
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

### Feature Intersection & Interpretability
Find SAE features that are active across all solutions and look them up on Neuronpedia.

In [ ]:
# @title Feature Intersection Analysis
print("SOCP: Active features per prompt")
for r, feats in zip(socp_results, socp_features):
    print(f"  \"{r['prompt'][:40]}...\" -> {len(feats)} features: {sorted(list(feats))[:10]}...")

print(f"\nQP: Active features per prompt")
for r, feats in zip(qp_results, qp_features):
    print(f"  \"{r['prompt'][:40]}...\" -> {len(feats)} features: {sorted(list(feats))[:10]}...")

# Intersection: features active in ALL SOCP solutions
socp_intersection = set.intersection(*socp_features) if socp_features else set()
qp_intersection = set.intersection(*qp_features) if qp_features else set()
all_intersection = socp_intersection | qp_intersection  # union of both intersections

print(f"\nSOCP features active in ALL prompts: {sorted(socp_intersection)}")
print(f"QP features active in ALL prompts: {sorted(qp_intersection)}")
print(f"Combined (union of intersections): {sorted(all_intersection)}")

# Neuronpedia links for GPT-2 Small JB SAEs
print(f"\n--- Neuronpedia Links (layer {layer}) ---")
for feat_idx in sorted(all_intersection):
    url = f"https://www.neuronpedia.org/gpt2-small/{layer}-res-jb/{feat_idx}"
    print(f"  Feature {feat_idx}: {url}")

---
## Part 2: Toxicity Steering (GPT-2 Small + Civil Comments)

In [ ]:
# @title Load Toxicity Data + Train Probe
toxicity_config = DataConfig(
    dataset_name="toxicity",
    hf_path="google/civil_comments",
    split="train",
    text_col="text",
    label_col="toxicity",
    toxicity_threshold=0.5,
    max_samples_per_class=500,
    probe_train_ratio=0.8,
    max_seq_len=128,
)

tox_data = load_dataset_splits(toxicity_config)
print(f"Toxicity -- Train: {len(tox_data['train_texts'])}, Test: {len(tox_data['test_texts'])}")

# Extract activations
tox_train_acts = extract_activations(tox_data['train_texts'], model_wrapper, layer, batch_size=8)
tox_test_acts = extract_activations(tox_data['test_texts'], model_wrapper, layer, batch_size=8)

tox_train_np = tox_train_acts.numpy()
tox_test_np = tox_test_acts.numpy()
tox_train_labels = np.array(tox_data['train_labels'])
tox_test_labels = np.array(tox_data['test_labels'])

# Train toxicity probe
tox_probe = LinearProbe(d_model=config.model.d_model, C=0.01)
tox_probe.fit(tox_train_np, tox_train_labels)
print(f"Toxicity probe -- train: {tox_probe.score(tox_train_np, tox_train_labels):.4f}, "
      f"test: {tox_probe.score(tox_test_np, tox_test_labels):.4f}")

tox_probe_w = tox_probe.weight_vector
tox_probe_b = tox_probe.bias

In [ ]:
# @title Toxicity Steering
toxicity_prompts = [
    "I think people who disagree are",
    "The problem with the government is that",
    "When someone makes me angry, I want to",
    "My honest opinion about this group is",
    "The reason everything is failing is because",
]

# SOCP
tox_socp = ConvexOptimalSteering(epsilon=5.0, tau=0.5, solver="SCS", prefilter_topk=2000)
tox_socp_results, tox_socp_features = run_steering_experiment(
    toxicity_prompts, tox_socp, model_wrapper, layer, tox_probe, tox_probe_w, tox_probe_b,
    D, target_class=1, alpha_values=alpha_values,
)
display_results(tox_socp_results, "SOCP -- Toxicity (epsilon=5.0)")

# QP
tox_qp = QPOptimalSteering(lam=1.0, tau=0.5, solver="SCS", prefilter_topk=2000, warm_start=True)
tox_qp_results, tox_qp_features = run_steering_experiment(
    toxicity_prompts, tox_qp, model_wrapper, layer, tox_probe, tox_probe_w, tox_probe_b,
    D, target_class=1, alpha_values=alpha_values,
)
display_results(tox_qp_results, "QP -- Toxicity (lambda=1.0)")

In [ ]:
# @title Toxicity: Probe Scores & Feature Analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, (name, results) in zip(axes, [("SOCP", tox_socp_results), ("QP", tox_qp_results)]):
    for r in results:
        scores = [r['generations'][a]['probe_score'] for a in alpha_values]
        ax.plot(alpha_values, scores, marker='o', label=r['prompt'][:30] + "...")
    ax.set_xlabel('Alpha')
    ax.set_ylabel('P(toxic)')
    ax.set_title(f'{name}: Toxicity Score vs Alpha')
    ax.legend(fontsize=8)
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

# Feature intersection
tox_socp_inter = set.intersection(*tox_socp_features) if tox_socp_features else set()
tox_qp_inter = set.intersection(*tox_qp_features) if tox_qp_features else set()
tox_all_inter = tox_socp_inter | tox_qp_inter

print(f"\nToxicity -- SOCP features in ALL prompts: {sorted(tox_socp_inter)}")
print(f"Toxicity -- QP features in ALL prompts: {sorted(tox_qp_inter)}")
print(f"\n--- Neuronpedia Links (layer {layer}) ---")
for feat_idx in sorted(tox_all_inter):
    url = f"https://www.neuronpedia.org/gpt2-small/{layer}-res-jb/{feat_idx}"
    print(f"  Feature {feat_idx}: {url}")

## Takeaways
- Both SOCP and QP find extremely sparse solutions (typically 3-15 active features out of ~24k)
- Higher alpha increases probe score shift but may degrade generation quality
- The QP formulation benefits from warm-starting across prompts
- Feature intersection analysis reveals which SAE features are consistently selected for a given attribute -- these can be inspected on Neuronpedia for interpretability